In [1]:
import pandas as pd
import dash
from dash import dcc, html
import plotly.graph_objects as go
from prophet import Prophet

In [2]:
# Load the dataset
file_path = "Transformed_Cancer_Incidence.csv"
df = pd.read_csv(file_path)

In [3]:
# Prepare Data for Prophet Model
def prepare_prophet_data(region_df):
    region_df = region_df[["Year", "Incidence"]].dropna()
    region_df.columns = ["ds", "y"]
    return region_df

In [4]:
# Train Prophet Models & Make Predictions
regions = df["region"].unique()
predictions = {}

for region in regions:
    region_data = df[df["region"] == region]
    prophet_df = prepare_prophet_data(region_data)

    model = Prophet()
    model.fit(prophet_df)

    future = model.make_future_dataframe(periods=5, freq='Y')
    forecast = model.predict(future)
    predictions[region] = forecast  # Store predictions


15:31:04 - cmdstanpy - INFO - Chain [1] start processing
15:31:12 - cmdstanpy - INFO - Chain [1] done processing
15:31:12 - cmdstanpy - INFO - Chain [1] start processing
15:31:20 - cmdstanpy - INFO - Chain [1] done processing
15:31:21 - cmdstanpy - INFO - Chain [1] start processing
15:31:21 - cmdstanpy - INFO - Chain [1] done processing
15:31:21 - cmdstanpy - INFO - Chain [1] start processing
15:31:30 - cmdstanpy - INFO - Chain [1] done processing
15:31:30 - cmdstanpy - INFO - Chain [1] start processing
15:31:38 - cmdstanpy - INFO - Chain [1] done processing
15:31:38 - cmdstanpy - INFO - Chain [1] start processing
15:31:47 - cmdstanpy - INFO - Chain [1] done processing
15:31:48 - cmdstanpy - INFO - Chain [1] start processing
15:31:56 - cmdstanpy - INFO - Chain [1] done processing
15:31:56 - cmdstanpy - INFO - Chain [1] start processing
15:31:56 - cmdstanpy - INFO - Chain [1] done processing
15:31:57 - cmdstanpy - INFO - Chain [1] start processing
15:31:57 - cmdstanpy - INFO - Chain [1]

In [5]:
# Dash App for Visualization
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Cancer Incidence Trends by Region"),
    dcc.Dropdown(
        id="region-dropdown",
        options=[{"label": r, "value": r} for r in regions],
        value=regions[0]
    ),
    dcc.Graph(id="trend-graph"),
])


In [6]:
@app.callback(
    dash.dependencies.Output("trend-graph", "figure"),
    [dash.dependencies.Input("region-dropdown", "value")]
)
def update_graph(selected_region):
    forecast = predictions[selected_region]
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=forecast["ds"], y=forecast["yhat"], mode="lines", name="Predicted Cases"))
    fig.update_layout(title=f"Cancer Incidence Trends - {selected_region}", xaxis_title="Year", yaxis_title="Incidence")
    
    return fig

# Ensure app runs correctly
if __name__ == '__main__':
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.